# Projeto final de Aprendizado de Máquina: análise e predição de dados clínicos
## UFMG - 2023/1 - Professor Adriano Veloso
### Luís Felipe Ramos Ferreira - 2019022553

Link para apresentação no youtube: link

# Introdução

blablabla

# Instalações e importações de bibliotecas

bla bla bla

In [ ]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

import warnings

# Base de dados

bla bla bla

# Implementação